## Tasseled Cap Transformation for atmospherical adjusted Landsat 8 imagery

**Author:** René Kopeinig<br>
**Description:** Tasseled Cap Transformation for Landsat 8 TOA imagery based on the scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance" by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index, greenness index and wetness index.

In [1]:
from IPython.display import Image
import ee, folium
ee.Initialize()

### Loading Landsat 8 TOA image 
Location: Oaxaca, Veracruz, Tabasco and Chiapas México

In [2]:
image = ee.Image('LANDSAT/LC8_L1T_TOA/LC80230482016032LGN00')

### Create function for Tasseled Cap Transformation

In [3]:
# Function
def tasseled_cap_transformation(image):
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA imagery
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

### Derive and visualize Tasseled Cap Transformation

In [4]:
tasseled_cap = tasseled_cap_transformation(image)

In [5]:
palette = ['FFFFFF','C0C0C0','808080','000000','FF0000','800000','FFFF00','808000','00FF00','008000','00FFFF','008080','0000FF','000080','FF00FF','800080']
palette = ','.join(palette)

### Brightness Index

In [6]:
region = tasseled_cap.geometry().getInfo()

vis_tct = {'min':-1,'max':2,'size':'800',
          'bands':['brightness'],
          'region':region}


mapid = tasseled_cap.getMapId(vis_tct)

map = folium.Map(location=[17.49,-95.09],zoom_start=8, height=500)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Tasseled Cap Transformation Brightness',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Greenness Index

In [7]:
vis_tct = {'min':-1,'max':2,'size':'800',
          'bands':['greenness'],
          'region':region}

mapid = tasseled_cap.getMapId(vis_tct)

map = folium.Map(location=[17.49,-95.09],zoom_start=8, height=500)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Tasseled Cap Transformation Greenness',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Wetness Index

In [8]:
vis_tct = {'min':-1,'max':2,'size':'800',
          'bands':['wetness'],
          'region':region}

mapid = tasseled_cap.getMapId(vis_tct)

map = folium.Map(location=[17.49,-95.09],zoom_start=8, height=500)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Tasseled Cap Transformation Wetness',
  ).add_to(map)

map.add_child(folium.LayerControl())
map